In [ ]:

# sessionInfo()
suppressPackageStartupMessages({
    library(DESeq2)
    library(dplyr)
    library(ggplot2)
    library(ggrepel)
    library(Matrix)
    library(ggpubr)
    library(RColorBrewer)
    library(cowplot)
    library(Matrix)
    library(stringr)
    library(DESeq2)
    library(edgeR)
    library(data.table)
    library(tximport)
    library(tidyr)
    library(readr)
    library(DESeq2)
    library(EnsDb.Hsapiens.v86)
    library(AnnotationHub)
    library(ggplot2)
    library(pheatmap)
    library(forcats)
    library(goseq)
    library(geneLenDataBase)
    library(org.Hs.eg.db)
    library(clusterProfiler)
    library(enrichplot)
    library(DOSE)
    library(ggpubr)
    library(TxDb.Hsapiens.UCSC.hg38.knownGene)
    library(tictoc)
})
# library(tximport)
# library(ggpubr)
# library(TxDb.Hsapiens.UCSC.hg38.knownGene)
TxDb= TxDb.Hsapiens.UCSC.hg38.knownGene


In [ ]:
packageVersion("DESeq2")

### Input File Names

In [ ]:
indir = '/nfs/lab/projects/islet_multiomics_stress_CoCl2/pipeline/bulk_islet_rna/bulk_RNA_salmon_v2023_02_17/quants_files_together/'
outdir = '/nfs/lab/projects/islet_multiomics_stress_CoCl2/pipeline/bulk_islet_rna/'

bulk_metadataFile = '/nfs/lab/projects/islet_multiomics_stress_CoCl2/pipeline/bulk_islet_rna/islet_bulk_rna_hg38.metadata.txt'


bulk_quant_file_suffix = '.quant.sf'

In [ ]:
up.pal <- brewer.pal(n = 6, name = "Reds")
down.pal <- brewer.pal(n = 6, name = "Blues")
# bulk_quant_file_dir = '/nfs/lab/projects/islet_multiomics_stress_CoCl2/pipeline/DESeq/input_files/bulk_rna_seq_islets_hg38/bulk_RNA_salmon_v2023_02_17/'
# 'nfs/lab/jnewsome/scripts/multiome_cluster_CoCl2_Fall2022/bulk/bulk_RNA_seq_CoCl2_trim_Salmon_quant_file_Name_Column.ipynb'

sampleNames = c("CO1_rep1c", "CO1_rep2c", "CO2_rep1c", "CO2_rep2c", "R395_CoCl2_1", 
                "R395_CoCl2_2", "R395_UNT", "UT1_rep1c", "UT1_rep2c", "UT2_rep1c")

#Gene info table
Gene.symbols.anno = "/nfs/lab/Luca/Assets/references/Salmon.index/Hg38.genecode.41/gencode.v41.metadata.HGNC"
GenCode.anno = "/nfs/lab/Luca/Assets/references/Salmon.index/Hg38.genecode.41/gencode.v41.annotation.gtf"
bulk_tx2gene_file = "/nfs/lab/Luca/Assets/references/Salmon.index/Hg38.genecode.41/genecode41.tx2gene"


### Read in Gene Annotation Files

In [ ]:
bulk_tx2gene = read.table(bulk_tx2gene_file, header = TRUE)
colnames(bulk_tx2gene) = c('TXNAME','GENEID')
# head(bulk_tx2gene)

Gene.symbols = read.table(Gene.symbols.anno, header = FALSE)
Gene.symbols = Gene.symbols[,-3]
colnames(Gene.symbols) = c("TXNAME", "Symbol")
# head(Gene.symbols)
# head(bulk_tx2gene)
# bulk_txtnames <- as.character(unlist(bulk_tx2gene$TXNAME))
# bulk_txtnames_1 <- str_extract(bulk_txtnames, ".*(?=\\.)")
# bulk_tx2gene$TXNAME <- bulk_txtnames_1
# bulk_geneIDs <-  as.character(unlist(bulk_tx2gene$GENEID))
# bulk_geneIDs_1 <- str_extract(bulk_geneIDs, ".*(?=\\.)")
# bulk_tx2gene$GENEID = bulk_geneIDs_1

Transcripts.Gene.symbols = merge(Gene.symbols, bulk_tx2gene, by = "TXNAME")
# Transcripts.Gene.symbols_noVer <- Transcripts.Gene.symbols
# bulk_txtnames <- as.character(unlist(Transcripts.Gene.symbols_noVer$GENEID))
# bulk_txtnames_1 <- str_extract(bulk_txtnames, ".*(?=\\.)")
# Transcripts.Gene.symbols_noVer$GENEID <- bulk_txtnames_1
head(Transcripts.Gene.symbols)


In [ ]:
tic()
gencode41Tablefile = '/nfs/lab/ref_to_organize/gencode_v41/gencode.v41.basic.annotation.gtf_converted_to_df_table.txt'
gencode41_orig = read.table(gencode41Tablefile)
toc()
gencode41_orig = gencode41_orig[,c('width', 'gene_id', 'gene_name', 'type')]

# head(gencode41_orig)


# table(gencodeTable$type)


gencode41_gene = gencode41_orig %>% dplyr::filter(type == 'gene')

gencode41_gene = unique(gencode41_gene)
gencode41_gene$type = NULL
# head(gencodeTable)
gencode41_gene = unique(gencode41_gene)
colorder = c('gene_id', 'gene_name', 'width', 'type')
gencode41 = gencode41_orig[,colorder]


colorder = c('gene_id', 'gene_name', 'width')
gencode41_gene = gencode41_gene[,colorder]

rownames(gencode41_gene) = gencode41_gene$gene_id
head(gencode41_gene)


### Read in and process metadata file

In [ ]:
bulk_metadata = read.table(bulk_metadataFile, header=T)
bulk_metadata


### Read in salmon files

In [ ]:
bulk_files = paste(indir, bulk_metadata$sample_label, bulk_quant_file_suffix, sep="")

names(bulk_files) <- bulk_metadata$sample_label
print(bulk_files)
bulk_txi <- tximport(bulk_files, type="salmon", tx2gene=bulk_tx2gene, ignoreAfterBar=TRUE, txOut=FALSE, ignoreTxVersion = FALSE)

In [ ]:
head(bulk_txi$counts)

head(bulk_txi$abundance)

head(bulk_txi$length)
head(bulk_txi$countsFromAbundance)


In [ ]:
counts_table = as.data.frame(bulk_txi$counts)
abundance_table = as.data.frame(bulk_txi$abundance)
length_table = as.data.frame(bulk_txi$length)

In [ ]:
head(counts_table)

In [ ]:
# Gene.symbols_sort = Transcripts.Gene.symbols
# Gene.symbols_sort$TXNAME = NULL
# Gene.symbols_sort = unique(Gene.symbols_sort)
# rownames(Gene.symbols_sort) = Gene.symbols_sort$GENEID
# head(Gene.symbols_sort)


In [ ]:
gene_ids_in_ref_table = rownames(Gene.symbols_sort)
gene_ids_in_counts_table = rownames(counts_table)
gene_ids_in_ref_table_gencode41_gene = rownames(gencode41_gene)


# length(gene_ids_in_ref_table)
# length(gene_ids_in_ref_table_gencode41_gene)
# length(gene_ids_in_counts_table)
# length(setdiff(gene_ids_in_counts_table, gene_ids_in_ref_table))
# length(setdiff(gene_ids_in_counts_table, gene_ids_in_ref_table_gencode41_gene))
gencode41_gene_sort = gencode41_gene
gencode41_gene_sort = gencode41_gene_sort[gene_ids_in_counts_table,]
counts_table$gene_name = gencode41_gene_sort$gene_name
counts_table$gene_width_from_gencode41 = gencode41_gene_sort$width
# head(counts_table)
# tail(counts_table)

In [ ]:
# counts_table$gene_width = NULL
head(counts_table)

In [ ]:
abundance_table$gene_name = gencode41_gene_sort$gene_name
length_table$gene_name = gencode41_gene_sort$gene_name
abundance_table$gene_width_from_gencode41 = gencode41_gene_sort$width
length_table$gene_width_from_gencode41 = gencode41_gene_sort$width

In [ ]:
outdir = '/nfs/lab/projects/islet_multiomics_stress_CoCl2/pipeline/bulk_islet_rna/'

tic('save txi file')



outfilename = paste0(outdir, 'bulk_islet_rna.hg38.counts_object_from_tximport_and_salmon.unannotated.unfiltered.unmerged.rds' )
saveRDS(bulk_txi, file = outfilename)

toc()





In [ ]:
tablename = paste0(outdir, 'bulk_islet_rna.hg38.counts_object_from_tximport_and_salmon.unfiltered.unmerged.Counts_table.txt')
tic()
write.table(counts_table, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()

tablename = paste0(outdir, 'bulk_islet_rna.hg38.counts_object_from_tximport_and_salmon.unfiltered.unmerged.Lengths_table.txt')
tic()
write.table(length_table, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()


tablename = paste0(outdir, 'bulk_islet_rna.hg38.counts_object_from_tximport_and_salmon.unfiltered.unmerged.Abundance_table.txt')
tic()
write.table(abundance_table, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()



#### Make DDS object

In [ ]:
dds  <- DESeqDataSetFromTximport(bulk_txi, colData = bulk_metadata, design= ~ Donor + Treatment)

In [ ]:
counts_from_dds = as.data.frame(counts(dds))
head(counts_from_dds)

In [ ]:
dim(counts_from_dds)

In [ ]:
gencode41_gene_sort2 = gencode41_gene
gencode41_gene_sort2 = gencode41_gene_sort2[rownames(counts_from_dds),]
counts_from_dds$gene_name = gencode41_gene_sort2$gene_name
counts_from_dds$gene_width_from_gencode41 = gencode41_gene_sort2$width
head(counts_from_dds)

In [ ]:
counts_from_dds$gene_id = rownames(counts_from_dds)
counts_from_dds2 = counts_from_dds
rownames(counts_from_dds2) = make.names(counts_from_dds2$gene_name,unique=TRUE)
annotations_table = counts_from_dds2
annotations_table$row_name = rownames(annotations_table)
annotations_table = annotations_table[,c('row_name', 'gene_name', 'gene_id', 'gene_width_from_gencode41')]
head(annotations_table)

In [ ]:
head(counts_from_dds2)

In [ ]:
counts_from_dds2$gene_name = NULL
counts_from_dds2$gene_id = NULL
counts_from_dds2$gene_width_from_gencode41 = NULL

In [ ]:
dds  = DESeqDataSetFromMatrix(
      countData = counts_from_dds2,
      bulk_metadata,
       design= ~ Donor + Treatment
    )


In [ ]:
counts_table$gene_width_from_gencode41 = NULL
abundance_table$gene_width_from_gencode41 = NULL
length_table$gene_width_from_gencode41 = NULL

In [ ]:
rowData(dds) = annotations_table


rownames(counts_table) = annotations_table$row_name
rownames(length_table) = annotations_table$row_name
rownames(abundance_table) = annotations_table$row_name
counts_table$gene_name = NULL
counts_table$gene_id = NULL
length_table$gene_name = NULL
length_table$gene_id = NULL
abundance_table$gene_name = NULL
abundance_table$gene_id = NULL
assays(dds)[["counts_from_tximport"]] <- as.matrix(counts_table)
assays(dds)[["length_from_tximport"]] <- as.matrix(length_table)
assays(dds)[["abundance_from_tximport"]] <- as.matrix(abundance_table)




In [ ]:
dds

In [ ]:
head(as.data.frame(rowData(dds)))

In [ ]:
ddsoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.ANNOTATED.countsfromTxImport.unfiltered.unmerged.dds.rds')
tic()
saveRDS(dds, file = ddsoutfilename)
toc()

## Filter out Genes with fewer than 2 reads per treatment


In [ ]:
unt_info = bulk_metadata %>% dplyr::filter(Treatment == 'Unt')
cocl2_info = bulk_metadata %>% dplyr::filter(Treatment == 'CoCl2')

untrows <-  as.character(as.factor(rownames(unt_info)))
cocl2rows <-  as.character(as.factor(rownames(cocl2_info)))

unt_counts <- counts_from_dds2[,untrows]
cocl2_counts <- counts_from_dds2[,cocl2rows]
geneNames <- rownames(counts_from_dds2)
    sumsGenes_cocl2 <- rowSums(cocl2_counts)
    sumsGenes_unt <- rowSums(unt_counts)

In [ ]:
sumsGenes_cocl2_df = as.data.frame(sumsGenes_cocl2)
sumsGenes_unt_df = as.data.frame(sumsGenes_unt)
sumsGenes_unt_df$dummy = 1

In [ ]:
head(sumsGenes_unt_df)

In [ ]:
sumsGenes_cocl2_df$cocl2 = sumsGenes_cocl2_df$sumsGenes_cocl2
sumsGenes = sumsGenes_cocl2_df
sumsGenes$unt = sumsGenes_unt_df$sumsGenes_unt
sumsGenes$sumsGenes_cocl2 = NULL
head(sumsGenes)

In [ ]:
sumsGenes_pass = sumsGenes %>% dplyr::filter(cocl2 >= 2 & unt >= 2)
dim(sumsGenes_pass)

In [ ]:
genes_pass = rownames(sumsGenes_pass)

In [ ]:
annotations_table_pass = annotations_table[genes_pass,]
counts_table_pass = counts_table[genes_pass,]
length_table_pass = length_table[genes_pass,]
abundance_table_pass = abundance_table[genes_pass,]
counts_from_dds2_pass = counts_from_dds2[genes_pass,]
dim(annotations_table_pass)

In [ ]:
dds_filt  = DESeqDataSetFromMatrix(
      countData = counts_from_dds2_pass,
      bulk_metadata,
       design= ~ Donor + Treatment
    )


In [ ]:
assays(dds_filt)[["counts_from_tximport"]] <- as.matrix(counts_table_pass)
assays(dds_filt)[["length_from_tximport"]] <- as.matrix(length_table_pass)
assays(dds_filt)[["abundance_from_tximport"]] <- as.matrix(abundance_table_pass)
rowData(dds_filt) = annotations_table_pass

In [ ]:
head(as.data.frame(rowData(dds_filt)))

In [ ]:
head(as.data.frame(counts(dds_filt)))

In [ ]:
ddsoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.ANNOTATED.FILT_2Plus_Reads_per_Treat.countsfromTxImport.unmerged.dds.rds')
tic()
saveRDS(dds_filt, file = ddsoutfilename)
toc()


## TPMs -- unmerged

In [ ]:
head(gencode41_gene)

In [ ]:
# Function to calculate TPM
calculate_tpm <- function(counts, gene_lengths) {
  # Calculate RPK (Reads Per Kilobase)
  rpk <- counts / (gene_lengths / 1e3)
  
  # Calculate per-million scaling factor
  scaling_factor <- colSums(rpk) / 1e6
  
  # Calculate TPM
  tpm <- t(t(rpk) / scaling_factor)
  
  return(tpm)
}



#### Unfiltered

In [ ]:
head(counts_from_dds2)
head(annotations_table)

In [ ]:

tic('calc tpm')
# Calculate TPM
tpm_unfilt <- calculate_tpm(counts_from_dds2, annotations_table$gene_width_from_gencode41)

toc()


# View the first few rows of the TPM matrix
head(tpm_unfilt)

#### Filt

In [ ]:
tpm_filt = tpm_unfilt[genes_pass,]

In [ ]:
head(tpm_filt)

In [ ]:
assays(dds)[["TPMs"]] <- as.matrix(tpm_unfilt)
assays(dds_filt)[["TPMs"]] <- as.matrix(tpm_filt)

In [ ]:
ddsoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.ANNOTATED.with_TPMs.countsfromTxImport.unfiltered.unmerged.dds.rds')
tic()
saveRDS(dds, file = ddsoutfilename)
toc()

ddsoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.countsfromTxImport.unmerged.dds.rds')
tic()
saveRDS(dds_filt, file = ddsoutfilename)
toc()


In [ ]:

tablename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.ANNOTATED.with_TPMs.countsfromTxImport.unfiltered.TPMs_table.txt')
tic()
write.table(tpm_unfilt, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()


tablename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.countsfromTxImport.unmerged.TPMs_table.txt')
tic()
write.table(tpm_filt, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()


## Merge Donor replicates 

In [ ]:
head(counts_from_dds2)

In [ ]:
bulk_metadata

In [ ]:



counts_from_dds_merge = counts_from_dds2
counts_from_dds_merge$CO1_rep_1c_2c = counts_from_dds_merge$CO1_rep1c + counts_from_dds_merge$CO1_rep2c
counts_from_dds_merge$UT1_rep_1c_2c = counts_from_dds_merge$UT1_rep1c + counts_from_dds_merge$UT1_rep2c
counts_from_dds_merge$CO2_rep_1c_2c = counts_from_dds_merge$CO2_rep1c + counts_from_dds_merge$CO2_rep2c
counts_from_dds_merge$R395_CoCl2_1_2 = counts_from_dds_merge$R395_CoCl2_1 + counts_from_dds_merge$R395_CoCl2_2


counts_from_dds_merge$CO1_rep1c = NULL
counts_from_dds_merge$CO1_rep2c = NULL
counts_from_dds_merge$UT1_rep1c = NULL
counts_from_dds_merge$UT1_rep2c = NULL
counts_from_dds_merge$CO2_rep1c = NULL
counts_from_dds_merge$CO2_rep2c = NULL
counts_from_dds_merge$R395_CoCl2_1 = NULL
counts_from_dds_merge$R395_CoCl2_2 = NULL

merged_sampleOrder = c('UT2_rep1c', 'R395_UNT', 'CO1_rep_1c_2c', 
                       'UT1_rep_1c_2c', 'CO2_rep_1c_2c', 'R395_CoCl2_1_2')
counts_from_dds_merge = counts_from_dds_merge[,merged_sampleOrder]
head(counts_from_dds_merge)

In [ ]:
dim(counts_from_dds_merge)

In [ ]:
bulk_metadata

In [ ]:
bulk_metadata$replicate_names = bulk_metadata$sample_label

In [ ]:
bulk_metadata_merge = bulk_metadata


# bulk_metadata_merge['CO1_rep_1c_2c',] = bulk_metadata_merge['CO1_rep1c',]
# bulk_metadata_merge[bulk_metadata_merge$sample_label == "CO1_rep_1c_2c", "sample_label"] <- "CO1_rep_1c_2c"
# bulk_metadata_merge[bulk_metadata_merge$sample_label == "CO1_rep_1c_2c", "replicate"] <- "rep1;rep2"
# bulk_metadata_merge[bulk_metadata_merge$sample_label == "CO1_rep_1c_2c", "replicate_names"] <- 'CO1_rep1c;CO1_rep2c'


new_row <- data.frame(sample_label = "CO1_rep_1c_2c", 
                      time = 24, 
                      replicate = "rep1;rep2", 
                      batch = "B",
                      condition = "CoCl2_24",
                      donor_batch = "R382_B",
                      Treatment = "CoCl2",
                      Donor = "R382",
                      replicate_names = "CO1_rep1c;CO1_rep2c"
                     )


bulk_metadata_merge <- rbind(bulk_metadata_merge, new_row)




new_row <- data.frame(sample_label = "UT1_rep_1c_2c", 
                      time = 24, 
                      replicate = "rep1;rep2", 
                      batch = "B",
                      condition = "Unt_24",
                      donor_batch = "R382_B",
                      Treatment = "Unt",
                      Donor = "R382",
                      replicate_names = "UT1_rep1c;UT1_rep2c"
                     )


bulk_metadata_merge <- rbind(bulk_metadata_merge, new_row)


new_row <- data.frame(sample_label = "CO2_rep_1c_2c", 
                      time = 24, 
                      replicate = "rep1;rep2", 
                      batch = "B",
                      condition = "CoCl2_24",
                      donor_batch = "R385_B",
                      Treatment = "CoCl2",
                      Donor = "R385",
                      replicate_names = "CO2_rep1c;CO2_rep2c"
                     )


bulk_metadata_merge <- rbind(bulk_metadata_merge, new_row)


new_row <- data.frame(sample_label = "R395_CoCl2_1_2", 
                      time = 24, 
                      replicate = "rep1;rep2", 
                      batch = "C",
                      condition = "CoCl2_24",
                      donor_batch = "R395_C",
                      Treatment = "CoCl2",
                      Donor = "R395",
                      replicate_names = "R395_CoCl2_1;R395_CoCl2_2"
                     )


bulk_metadata_merge <- rbind(bulk_metadata_merge, new_row)




rownames(bulk_metadata_merge) = bulk_metadata_merge$sample_label
bulk_metadata_merge = bulk_metadata_merge[merged_sampleOrder,]

bulk_metadata_merge

In [ ]:
dds_mergedReplicates  = DESeqDataSetFromMatrix(
      countData = counts_from_dds_merge,
      bulk_metadata_merge,
       design= ~ Donor + Treatment
    )
rowData(dds_mergedReplicates) = annotations_table
dds_mergedReplicates

## TPMs -- merged

In [ ]:
counts_from_dds_merge2 = counts_from_dds_merge
counts_from_dds_merge2$gene_name = NULL
counts_from_dds_merge2$gene_id = NULL
counts_from_dds_merge2$gene_width_from_gencode41 = NULL


In [ ]:
head(unlist(as.list(annotations_table$gene_width_from_gencode41)))

In [ ]:
# annotations_table

In [ ]:

tic('calc tpm')
# Calculate TPM
tpm_merged_unfilt <- calculate_tpm(counts_from_dds_merge2, unlist(as.list(annotations_table$gene_width_from_gencode41)))

toc()


# View the first few rows of the TPM matrix
head(tpm_merged_unfilt)

In [ ]:
tpm_merged_filt = tpm_merged_unfilt[genes_pass,]

In [ ]:
counts_from_dds_merge_filt = counts_from_dds_merge2[genes_pass,]

In [ ]:
dds_mergedReplicates_filt  = DESeqDataSetFromMatrix(
      countData = counts_from_dds_merge_filt,
      bulk_metadata_merge,
       design= ~ Donor + Treatment
    )
rowData(dds_mergedReplicates_filt) = annotations_table_pass
dds_mergedReplicates_filt

In [ ]:
assays(dds_mergedReplicates)[["TPMs"]] <- as.matrix(tpm_merged_unfilt)
assays(dds_mergedReplicates_filt)[["TPMs"]] <- as.matrix(tpm_merged_filt)

In [ ]:
ddsoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.with_TPMs.countsfromTxImport.unfiltered.dds.rds')
tic()
saveRDS(dds_mergedReplicates, file = ddsoutfilename)
toc()

ddsoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.countsfromTxImport.dds.rds')
tic()
saveRDS(dds_mergedReplicates_filt, file = ddsoutfilename)
toc()


In [ ]:
head(annotations_table)

In [ ]:

tablename = paste0(outdir, 'islet_bulk_rna_hg38.gene_annotations_from_Gencode41.unfiltered.txt')
tic()
write.table(annotations_table, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()


tablename = paste0(outdir, 'islet_bulk_rna_hg38.gene_annotations_from_Gencode41.FILT_2Plus_Reads_per_Treat.txt')
tic()
write.table(annotations_table_pass, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()



tablename = paste0(outdir, 'islet_bulk_rna_hg38.sample_metadata.MergedReplicates.txt')
tic()
write.table(bulk_metadata_merge, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()



In [ ]:
counts_final = counts_from_dds2
counts_merged_final = counts_from_dds_merge2
tpm_final = as.data.frame(tpm_unfilt)
tpm_merged_final = as.data.frame(tpm_merged_unfilt)
# annotations_table

counts_filt_final = counts_from_dds2_pass
counts_merged_filt_final = counts_from_dds_merge_filt
tpm_filt_final =   as.data.frame(tpm_filt)
tpm_merged_filt_final =   as.data.frame(tpm_merged_filt)
# annotations_table_pass

In [ ]:
counts_final$gene_name =  annotations_table$gene_name
counts_final$gene_id =  annotations_table$gene_id
counts_final$gene_width_from_gencode41 =  annotations_table$gene_width_from_gencode41

counts_merged_final$gene_name =  annotations_table$gene_name
counts_merged_final$gene_id =  annotations_table$gene_id
counts_merged_final$gene_width_from_gencode41 =  annotations_table$gene_width_from_gencode41

tpm_final$gene_name =  annotations_table$gene_name
tpm_final$gene_id =  annotations_table$gene_id
tpm_final$gene_width_from_gencode41 =  annotations_table$gene_width_from_gencode41


tpm_merged_final$gene_name =  annotations_table$gene_name
tpm_merged_final$gene_id =  annotations_table$gene_id
tpm_merged_final$gene_width_from_gencode41 =  annotations_table$gene_width_from_gencode41



counts_filt_final$gene_name =  annotations_table_pass$gene_name
counts_filt_final$gene_id =  annotations_table_pass$gene_id
counts_filt_final$gene_width_from_gencode41 =  annotations_table_pass$gene_width_from_gencode41

counts_merged_filt_final$gene_name =  annotations_table_pass$gene_name
counts_merged_filt_final$gene_id =  annotations_table_pass$gene_id
counts_merged_filt_final$gene_width_from_gencode41 =  annotations_table_pass$gene_width_from_gencode41

tpm_merged_filt_final$gene_name =  annotations_table_pass$gene_name
tpm_merged_filt_final$gene_id =  annotations_table_pass$gene_id
tpm_merged_filt_final$gene_width_from_gencode41 =  annotations_table_pass$gene_width_from_gencode41


tpm_filt_final$gene_name =  annotations_table_pass$gene_name
tpm_filt_final$gene_id =  annotations_table_pass$gene_id
tpm_filt_final$gene_width_from_gencode41 =  annotations_table_pass$gene_width_from_gencode41





In [ ]:
head(counts_final)
head(counts_merged_final)
head(tpm_final)
head(tpm_merged_final)

head(counts_filt_final)
head(counts_merged_filt_final)
head(tpm_filt_final)
head(tpm_merged_filt_final)



In [ ]:
tablename = paste0(outdir, 'bulk_islet_rna.hg38.Counts.unfiltered.unmerged.txt')
tic()
write.table(counts_final, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()

tablename = paste0(outdir, 'bulk_islet_rna.hg38.Counts.unfiltered.MergedReplicates.txt')
tic()
write.table(counts_merged_final, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()

tablename = paste0(outdir, 'bulk_islet_rna.hg38.TPMs.unfiltered.unmerged.txt')
tic()
write.table(tpm_final, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()

tablename = paste0(outdir, 'bulk_islet_rna.hg38.TPMs.unfiltered.MergedReplicates.txt')
tic()
write.table(tpm_merged_final, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()



tablename = paste0(outdir, 'bulk_islet_rna.hg38.Counts.FILT_min2_Reads_per_Treat.unmerged.txt')
tic()
write.table(counts_filt_final, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()

tablename = paste0(outdir, 'bulk_islet_rna.hg38.Counts.FILT_min2_Reads_per_Treat.MergedReplicates.txt')
tic()
write.table(counts_merged_filt_final, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()

tablename = paste0(outdir, 'bulk_islet_rna.hg38.TPMs.FILT_min2_Reads_per_Treat.unmerged.txt')
tic()
write.table(tpm_filt_final, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()

tablename = paste0(outdir, 'bulk_islet_rna.hg38.TPMs.FILT_min2_Reads_per_Treat.MergedReplicates.txt')
tic()
write.table(tpm_merged_filt_final, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()



In [ ]:

tablename = paste0(outdir, 'bulk_islet_rna.hg38.TPMs.unfiltered.unmerged.txt')
tic()
tpm_final = read.table( tablename)
toc()

tablename = paste0(outdir, 'bulk_islet_rna.hg38.TPMs.unfiltered.MergedReplicates.txt')
tic()
tpm_merged_final = read.table( tablename)
toc()




In [ ]:
head(tpm_final)

In [ ]:

tablename = paste0(outdir, 'bulk_islet_rna.hg38.TPMs.unfiltered.MergedReplicates.csv')
tic()
write.table(tpm_merged_final, tablename, quote=FALSE, sep=',',  col.names = TRUE, row.names=TRUE)
toc()



tablename = paste0(outdir, 'bulk_islet_rna.hg38.TPMs.unfiltered.unmerged.csv')
tic()
write.table(tpm_final, tablename, quote=FALSE, sep=',', col.names = TRUE, row.names=TRUE)
toc()
